In [59]:
%matplotlib inline

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

In [156]:
movies = pd.read_csv('data/IMDbMovies.csv')
movies.columns

Index(['movie title', 'Run Time', 'Rating', 'User Rating', 'Generes',
       'Overview', 'Plot Keyword', 'Director', 'Top 5 Casts', 'Writer', 'year',
       'path'],
      dtype='object')

In [157]:
def txt_to_int(txt):
    end = txt[-1]
    if end.isdigit():
        return int(txt)

    factor = 1
    if end == 'M':
        factor = 1e6
    elif end == 'K':
        factor = 1e3

    return int(factor * float(txt[:-1]))

def to_float(txt):
    try:
        return float(txt)
    except:
        return np.nan
    
def to_date(txt):
    try:
        return txt[1:]
    except:
        return txt

def preprocess(data : pd.DataFrame, dropna=True):
    df = pd.DataFrame()
    df['title'] = data['movie title']
    df['votes'] = data['User Rating'].apply(txt_to_int)
    df['rating'] = data['Rating'].apply(to_float) * np.log(df['votes'])

    df['writer'] = data['Writer']
    df['director'] = data['Director']

    to_expression = lambda x: eval(x)
    df['actors'] = data['Top 5 Casts'].apply(to_expression)
    df['generes'] = data['Generes'].apply(to_expression)
    df['plotwords'] = data['Plot Keyword'].apply(to_expression)
    df['year'] = data['year'].apply(to_date)

    if dropna:
        # NaN occurs when not enough people have watched the film
        # OR
        # enough data wasn't collected
        df.dropna(inplace=True)

    return df



In [158]:
df = preprocess(movies)

/home/javkt/anaconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


,title,votes,rating,writer,director,actors,generes,plotwords,year
496,Napoleon,7800,73.487408,Abel Gance,Abel Gance,"[Albert Dieudonné, Vladimir Roudenko, Edmond V...","[Biography, Drama, History]","[military uniform, french politics, french rep...",NaN
562,I Saw the Devil,127000,91.665150,Park Hoon-jung,Jee-woon Kim,"[Jee-woon Kim, Lee Byung-hun, Choi Min-sik, Je...","[Action, Crime, Drama]","[revenge, decapitation, attempted rape, tracki...",NaN
1060,The Killer,48000,84.075859,John Woo,John Woo,"[Chow Yun-Fat, Danny Lee, Sally Yeh, John Woo,...","[Action, Crime, Drama]","[hong kong, hitman, gunfight, police, 1980s, c...",NaN
1061,The Killer,14,22.431987,Ji-Woong Nam,Jae-Hoon Choi,"[Jang Hyuk, Seo-young Lee, Bang Eun-Jung, Jae-...",[Action],[one person army action],NaN
1062,The Killer,2700,47.406042,Marcelo Galvão,Marcelo Galvão,"[Maria de Medeiros, Joe Ochman, Diogo Morgado,...","[Action, Western]","[male nudity, written by director, 1940s, nort...",NaN
2739,The Tiger,5600,62.139758,Park Hoon-jung,Park Hoon-jung,"[Kim Hong-pa, Man-sik Jeong, Seok-won Jeong, P...","[Action, Adventure, Drama]","[occupation, hunter, tiger, japanese occupatio...",NaN
4288,Cyborg Nemesis: The Dark Rift,87,37.067037,Cynthia Curnan,Albert Pyun,"[Sasha Mitchell, Olivier Gruner, Vincent Klyn,...","[Action, Sci-Fi]","[cyborg, prequel, stasis, marine, laboratory, ...",NaN
5741,Apaharan,4800,62.725147,Prakash Jha,Prakash Jha,"[Shridhar Raghavan, Manoj Tyagi, Ajay Devgn, N...","[Action, Crime, Drama]","[man slaps a man, kidnapping, bihar, corruptio...",NaN
11338,The Occupant,17000,62.342199,Àlex Pastor,David Pastor,"[Àlex Pastor, David Pastor, Javier Gutiérrez, ...","[Adventure, Drama, Horror]","[intruder, executive, losing a job, berserk, j...",NaN
12915,Mothers' Instinct,1500,47.535933,Barbara Abel,Olivier Masset-Depasse,"[Giordano Gederlini, Olivier Masset-Depasse, V...","[Drama, Mystery, Thriller]","[female friends, paranoia, death of a child, d...",NaN
